In [1]:
import pandas as pd
import numpy as np
from keras import callbacks
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
inputs = pd.read_csv("../BreastCancer/entradas_breast.csv")
outputs = pd.read_csv("../BreastCancer/saidas_breast.csv")

In [3]:
# define callback
class Callback(callbacks.Callback):
    SHOW_NUMBER = 25
    
    epoch = 0

    # gives epoch feedback each 25 epochs
    def on_epoch_end(self, epoch, logs=None):
        self.epoch = epoch
        if (self.epoch % 25 == 0):
            print('Epoch: ' + str(self.epoch) + ' loss: ' + str(logs['loss']))

In [4]:
def createNetwork(
        optimizer,
        loss,
        kernel_initializer,
        activation,
        neurons
    ):
    classifier = Sequential()
    classifier.add(Dense(
        units=neurons,
        activation=activation,
        kernel_initializer=kernel_initializer,
        input_dim=30
    ))
    # units = (number_of_features + number_of_output_neurons)/2
    
    classifier.add(Dropout(0.2))
    
    classifier.add(Dense(
        units=neurons,
        activation=activation,
        kernel_initializer=kernel_initializer,
    ))

    classifier.add(Dropout(0.2))

    classifier.add(Dense(
        units=1,
        activation='sigmoid'  # critical parameter
    ))

    classifier.compile(
        optimizer=optimizer,
        # optimizer = 'adam'
        loss=loss,
        metrics=['binary_accuracy']
    )

    return classifier


In [5]:
classifier = createNetwork('adam', 'binary_crossentropy', 'normal', 'relu', '8')

In [6]:
classifier.fit(inputs, outputs, batch_size=10, epochs=100, use_multiprocessing=True, verbose=0, callbacks=[Callback()])

Epoch: 0 loss: 1.207785725593567
Epoch: 25 loss: 0.35102003812789917
Epoch: 50 loss: 0.2726804316043854
Epoch: 75 loss: 0.2601933479309082


In [7]:
breast_data = np.array([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]])

prediction = classifier.predict(breast_data)
prediction = (prediction > 0.5)
print(prediction)

1/1 [==============================] - 0s 198ms/step
[[ True]]


In [11]:
jsonClassifier = classifier.to_json()
with open('Models/breast_cancer_model.json', 'w') as json_file:
    json_file.write(jsonClassifier)

classifier.save_weights('Models/breast_cancer_model.h5')
